In [18]:
import os
import numpy as np
import pandas as pd
import time
from utm import *
from tqdm import tqdm, tqdm_pandas
from osgeo import osr
import coordTransform


In [33]:
DATE = '01' #填写01~30
DAY_BEGIN = '07:30:00'
DAY_END = '09:30:00'
day_begin_name = 730
day_end_name = 930

ORDER_FILE_PATH_WIN = f'F:/大学/第40期PRP/交通订单数据/traffic_data/gps_201611{DATE}.csv'
DST_DIR_PATH = 'F:/大学/第40期PRP/特征提取/坐标转换后的数据/'

time1 = '201611' + DATE + ' ' + DAY_BEGIN
time2 = '201611' + DATE + ' ' + DAY_END
stamp1 = time.mktime(time.strptime(time1, '%Y%m%d %H:%M:%S'))
stamp2 = time.mktime(time.strptime(time2, '%Y%m%d %H:%M:%S'))



In [20]:

#导入原地理数据
print(f'正在导入数据：gps_{DATE}.csv')
df_in = pd.read_csv(ORDER_FILE_PATH_WIN, header = None) #注意我此处使用的是移动硬盘的地址
df_in.columns = ['driver_ID', 'order_ID', 'timestamp', 'lon', 'lat']
df_in.timestamp = df_in.timestamp + 8*3600
print(f'已导入数据：gps_{DATE}.csv')

正在导入数据：gps_01.csv
已导入数据：gps_01.csv


In [34]:

## 只取预设时间区间内的数据
df = df_in[(df_in['timestamp'] >= stamp1)&(df_in['timestamp'] < stamp2)].reset_index(drop = True)

# 将空间坐标转换为WGS-84(耗时会很长)
xy = df[['lon','lat']].apply(lambda x: coordTransform.gcj02_to_wgs84(x[0],x[1])[:2], axis = 1)
df['lon'] = [x[0] for x in xy]
df['lat'] = [x[1] for x in xy]
print (f'{DATE} 已生成WGS-84')

# 再把WGS-84转换为UTM平面直角系(保留WGS-84数据)
wgs84 = osr.SpatialReference()
wgs84.ImportFromEPSG(4326)
# 2.Pseudo-Mercator
inp = osr.SpatialReference()
inp.ImportFromEPSG(3857)
# 3.定义坐标变换映射
transformation = osr.CoordinateTransformation(wgs84, inp)
# 4.转换原数据的坐标
print(f'{DATE} 正在转换原数据坐标')
xy = df[['lon','lat']].apply(lambda x: transformation.TransformPoint(x[0],x[1])[:2], axis = 1)
print(f'{DATE} 已转换原数据坐标')
# 5.写入df
df['x'] = [x[0] for x in xy]
df['y'] = [x[1] for x in xy]
print (f'{DATE} 已生成UTM, 当前数据条数：',len(df))


01 已生成WGS-84
01 正在转换原数据坐标
01 已转换原数据坐标
01 已生成UTM, 当前数据条数： 654744


In [35]:

df.to_csv(DST_DIR_PATH + f'201611{DATE}-{day_begin_name}-{day_end_name}.csv', index = False)
print(f'已写入数据:{DST_DIR_PATH}')

已写入数据:F:/大学/第40期PRP/特征提取/坐标转换后的数据/


In [36]:
df1 = pd.read_csv(DST_DIR_PATH + f'201611{DATE}-{day_begin_name}-{day_end_name}.csv')
df1

,driver_ID,order_ID,timestamp,lon,lat,x,y
0,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958895,104.100935,30.716003,1.158846e+07,3.595921e+06
1,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958898,104.100935,30.716003,1.158846e+07,3.595921e+06
2,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958901,104.100935,30.716003,1.158846e+07,3.595921e+06
3,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958904,104.100935,30.716003,1.158846e+07,3.595921e+06
4,rjzql9aaf.vmnm1qurudumjjefe@Glpb,ldwov8koc.Bxva9mCurmAtmkkeg4zjrh,1477958907,104.100935,30.716003,1.158846e+07,3.595921e+06
...,...,...,...,...,...,...,...
654739,iixwyccbh7vrvm6rsxpnpmniidabApyd,hnpunhlkb_vpCg0fDzucxlonjkfaxknc,1477960748,104.041940,30.726311,1.158190e+07,3.597256e+06
654740,iixwyccbh7vrvm6rsxpnpmniidabApyd,hnpunhlkb_vpCg0fDzucxlonjkfaxknc,1477960751,104.041721,30.726071,1.158187e+07,3.597225e+06
654741,iixwyccbh7vrvm6rsxpnpmniidabApyd,hnpunhlkb_vpCg0fDzucxlonjkfaxknc,1477960754,104.041461,30.725812,1.158184e+07,3.597192e+06
654742,iixwyccbh7vrvm6rsxpnpmniidabApyd,hnpunhlkb_vpCg0fDzucxlonjkfaxknc,1477960757,104.041162,30.725593,1.158181e+07,3.597163e+06
